# Задание по теме «Коллаборативная фильтрация»


* используйте данные MovieLens 1M
* можно использовать любые модели из пакета
* получите RMSE на тестовом сете 0.87 и ниже

In [2]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 8.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-macosx_10_9_x86_64.whl size=766565 sha256=26d1d96b7bdaea339cf2987ea70bda878eb49b9ede67ca43cef25d1b953116f1
  Stored in directory: /Users/glebmutovin/Library/Caches/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
You should consider upgrading via the '/Users/glebmutovin/Downloads/ENTER/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [56]:
movies = pd.read_table('movies.dat', names=['MovieID','Title','Genres'], sep='::')

<ipython-input-56-8e571ecc3214>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('movies.dat', names=['MovieID','Title','Genres'], sep='::')


In [57]:
users = pd.read_table('users.dat', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], sep='::')
ratings = pd.read_table('ratings.dat', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], sep='::')

<ipython-input-57-70d47cd7abe8>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table('users.dat', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], sep='::')
<ipython-input-57-70d47cd7abe8>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table('ratings.dat', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], sep='::')


In [58]:
movies_with_ratings = movies.join(ratings.set_index('MovieID'), on='MovieID').reset_index(drop=True)

In [59]:
movies_with_ratings.dropna(inplace=True)

In [41]:
movies_with_ratings

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,9.788243e+08
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,9.782370e+08
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,9.782335e+08
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,9.782260e+08
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,9.782265e+08
...,...,...,...,...,...,...
1000381,3952,"Contender, The (2000)",Drama|Thriller,5812.0,4.0,9.920721e+08
1000382,3952,"Contender, The (2000)",Drama|Thriller,5831.0,3.0,9.862231e+08
1000383,3952,"Contender, The (2000)",Drama|Thriller,5837.0,4.0,1.011903e+09
1000384,3952,"Contender, The (2000)",Drama|Thriller,5927.0,1.0,9.798525e+08


In [60]:
df_for_rs = pd.DataFrame(
    {
    'uid': movies_with_ratings.UserID,
    'iid': movies_with_ratings.Title,
    'rating': movies_with_ratings.Rating  
    }
)

In [61]:
df_for_rs

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0
...,...,...,...
1000381,5812.0,"Contender, The (2000)",4.0
1000382,5831.0,"Contender, The (2000)",3.0
1000383,5837.0,"Contender, The (2000)",4.0
1000384,5927.0,"Contender, The (2000)",1.0


In [62]:
reader = Reader(rating_scale=(1.0, 5.0))
dataset = Dataset.load_from_df(df_for_rs, reader)

In [63]:
trainset, testset = train_test_split(dataset, test_size=.15)

In [64]:
algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [65]:
test_pred = algo.test(testset)

In [66]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8596


0.8596015050464502